In [1]:
# Connection parameters
host = '35.228.34.114'
user= 'postgres'
password='P@ssw0rd!'
database='postgres'  # the name of the batabase we are connecting
port = "5432"

In [2]:
import sqlalchemy
import psycopg2
import pandas as pd
import sqlite3
import pandas as pd
from io import StringIO
import csv
from psycopg2 import extras
import numpy as np

In [3]:
# result (accurancy)

In [4]:
df = pd.read_csv('C:/Users/Alla/Desktop/苏黎世/第三学期/product/backend/data/dataprep_v2.csv')

In [5]:
cf = pd.read_csv('C:/Users/Alla/Desktop/苏黎世/第三学期/product/backend/data/final.csv')

In [6]:
# copy market share prediction
if len(df) == len(cf):
    df['ms_pred'] = cf['market_share_pred']
else:
    print("Error: The DataFrames do not have the same length.")

In [7]:
# calculate RMSE for each itinerary (same Orig, Dest)

In [8]:
# calculate RMSE
def rmse(group):
    return np.sqrt(((group['ms_pred'] - group['market_share']) ** 2).mean())

In [9]:
# Calculate RMSE for each itinerary
itinerary_rmse = df.groupby(['Orig', 'Dest']).apply(rmse)

In [10]:
# Map RMSE back to the original DataFrame
df['accuracy'] = df.apply(lambda row: itinerary_rmse[row['Orig'], row['Dest']], axis=1)

In [11]:
# check
print(df)

        Unnamed: 0  Orig  con1  con2  Dest  op_flight1  op_flight2  \
0                0    26    64   239   181         732        2624   
1                1    26    64   239   181         732        2624   
2                2    26    64   239   181         732        1748   
3                3    26    64   239   181         732        2624   
4                4    26    64   239   181         732        2624   
...            ...   ...   ...   ...   ...         ...         ...   
522801      522801    41   148   239    26          11        3060   
522802      522802    41    26   239    26         493         210   
522803      522803     1    41   239    26        7549        3114   
522804      522804     1     1     1    26         295        2221   
522805      522805     1     1    65    26         411        2583   

        op_flight3  depDay  elaptime  ...  cluster  TOT_pax  market_share  \
0             1101       2       535  ...      111     0.00          0.00   
1    

In [12]:
# Create a connection to the database
conn = psycopg2.connect(
    host=host,
    database=database,
    user=user,
    password=password,
    port=port
)

In [13]:
# Create a cursor object
cur = conn.cursor()

In [14]:
# insert accurancy value
df_selected = df[['accuracy', 'TOT_pax']]
df_selected['accuracy'] = df_selected['accuracy'].astype(float)

C:\Users\Alla\AppData\Local\Temp\ipykernel_22728\3525411174.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['accuracy'] = df_selected['accuracy'].astype(float)


In [15]:
# Retrieve all ids from the recommendation table
cur.execute("SELECT id FROM marketshare ORDER BY id")
marketshare_ids = cur.fetchall()

In [16]:
if len(df_selected) <= len(marketshare_ids):
    # Assign the itinerary_input_id values to the df_selected DataFrame
    df_selected['marketshare_id'] = [id_tuple[0] for id_tuple in marketshare_ids[:len(df_selected)]]

    # Reorder the DataFrame columns to match the table schema
    df_selected = df_selected[['marketshare_id', 'TOT_pax', 'accuracy']]
    # Convert the DataFrame to an in-memory buffer for copying to SQL
    buffer = StringIO()
    df_selected.to_csv(buffer, index=False, header=False) 
    # Use na_rep to replace NaN with \N in the CSV
    buffer.seek(0)

    # Use copy_from to copy the data to the database
    cur.copy_from(buffer, 'result', sep=',', columns=('marketshare_id', 'TOT_pax', 'accuracy'))
    conn.commit()
else:
    raise ValueError("There are more entries than fk_constrain available.")

C:\Users\Alla\AppData\Local\Temp\ipykernel_22728\4170759907.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['marketshare_id'] = [id_tuple[0] for id_tuple in marketshare_ids[:len(df_selected)]]


In [17]:
# Close the cursor and connection
cur.close()
conn.close()